# 🧪 Notebook Thử Nghiệm Chatbot

Notebook này dùng để thử nghiệm các thành phần của chatbot học tập.

## 1. Import thư viện và setup

In [ ]:
import sys
import os

# Thêm thư mục gốc vào path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

from src.helper import (
    load_all_pdfs,
    split_text_into_chunks,
    create_embeddings,
    create_vector_store_faiss,
    load_vector_store_faiss,
    create_chatbot,
    ask_question
)
from src.prompt import prompt_template, welcome_message

print("✅ Import thành công!")
print(welcome_message)

## 2. Load và xử lý PDF

In [ ]:
# Load tất cả PDF từ thư mục data
documents = load_all_pdfs("../data")

print(f"\n📊 Tổng số documents: {len(documents)}")
for i, doc in enumerate(documents, 1):
    print(f"{i}. {doc['source']}: {len(doc['text'])} ký tự")

## 3. Chia nhỏ text thành chunks

In [ ]:
# Split documents thành chunks
chunks = split_text_into_chunks(documents, chunk_size=1000, chunk_overlap=200)

print(f"\n📦 Tổng số chunks: {len(chunks)}")
print(f"\n📄 Ví dụ chunk đầu tiên:")
print(f"Source: {chunks[0]['source']}")
print(f"Chunk ID: {chunks[0]['chunk_id']}")
print(f"Nội dung (100 ký tự đầu): {chunks[0]['text'][:100]}...")

## 4. Tạo embeddings và vector store

In [ ]:
# Tạo embeddings (sử dụng HuggingFace - miễn phí)
embeddings = create_embeddings(use_openai=False)

# Tạo FAISS vector store
vector_store = create_vector_store_faiss(chunks, embeddings, index_path="../faiss_index")

print("\n✅ Vector store đã được tạo thành công!")

## 5. Thử nghiệm tìm kiếm

Test khả năng tìm kiếm thông tin liên quan từ vector store

In [ ]:
# Test tìm kiếm
query = "phóng xạ nhiệt là gì"
results = vector_store.similarity_search(query, k=3)

print(f"🔍 Tìm kiếm: '{query}'")
print(f"\n📚 Tìm thấy {len(results)} kết quả liên quan:\n")

for i, doc in enumerate(results, 1):
    print(f"--- Kết quả {i} ---")
    print(f"Source: {doc.metadata['source']}")
    print(f"Nội dung: {doc.page_content[:200]}...")
    print()

## 6. Tạo và test chatbot

⚠️ **Lưu ý:** Cần có OpenAI API key trong file `.env` để chạy phần này!

In [ ]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Tạo chatbot
qa_chain = create_chatbot(vector_store, prompt_template)

if qa_chain:
    print("✅ Chatbot đã sẵn sàng!")
else:
    print("⚠️ Không thể tạo chatbot - thiếu OpenAI API key")
    print("💡 Vui lòng thêm OPENAI_API_KEY vào file .env")

## 7. Đặt câu hỏi cho chatbot

In [ ]:
if qa_chain:
    # Test câu hỏi
    question = "Phóng xạ nhiệt là gì?"
    
    print(f"❓ Câu hỏi: {question}\n")
    response = ask_question(qa_chain, question)
    
    print(f"🤖 Trả lời:\n{response['answer']}\n")
    
    if response['sources']:
        print(f"📚 Nguồn tài liệu:")
        for source in response['sources']:
            print(f"  • {source['source']}")
else:
    print("⚠️ Chatbot chưa được khởi tạo. Vui lòng chạy cell trước đó.")

## 8. Test với nhiều câu hỏi

In [ ]:
if qa_chain:
    # Danh sách câu hỏi test
    test_questions = [
        "Các loại phóng xạ khí là gì?",
        "Phóng xạ hạt nhân có ứng dụng gì?",
        "Giải thích về chu trình phân rã phóng xạ",
    ]
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n{'='*60}")
        print(f"📝 Câu hỏi {i}: {question}")
        print(f"{'='*60}")
        
        response = ask_question(qa_chain, question)
        print(f"\n🤖 Trả lời:\n{response['answer']}")
        
        if response['sources']:
            print(f"\n📚 Nguồn: {', '.join([s['source'] for s in response['sources']])}")
else:
    print("⚠️ Chatbot chưa được khởi tạo. Vui lòng chạy cell trước đó.")